# Hexagon temperature blanket plotter

This plotter is designed to help visualize the layout of a temperature blanket, with each hexagon representing a single day of the year. The hexagons are arranged diagonally, starting from the top-left corner. The last hexagon is in the lower right corner.

Each hexagon can be colored in one or multiple COLOR_CHART, depending on the design. This notebook includes three layout styles:

- **Solid Hexagons**: The entire hexagon is filled with a color representing the average temperature of the day.
- **Split Diagonal Hexagons**: The top half of the hexagon shows the day's maximum temperature, while the bottom half shows the minimum.
- **Nested Hexagons**: A smaller inner hexagon represents the minimum temperature, surrounded by some outer hexagon indicating the maximum temperature.

Sections marked with `CHANGE HERE` can be customized.

To use this plotter, you’ll need a data file containing the minimum, maximum, and average temperatures for each day - in that exact order. Each line in the file should represent one day, with the three values separated accordingly:

```csv
min_temp,max_temp,avg_temp
10,20,15
23,28,25
1,7,6.8
...
```

In this example, 2 is the minimum, 10 is the maximum, and 6 is the average temperature.

So, let's start plotting :)

1. Import required libraries

In [35]:
import json
import csv
from collections import defaultdict
from functools import reduce
from pathlib import Path as SysPath

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.patches import PathPatch, Path, RegularPolygon
import numpy as np
import math

2. Define the color chart

  Each entry in the chart maps a temperature range to a specific color, using the following format:
  ```python
  (<start:range>, <end_range>): <color>,
  ```

  - <start_range> and <end_range> define the temperature interval (inclusive of the start, inclusive of the end).

  - <color> can have the following formats:
    - RGB or RGBA, e.g.: `(0.5, 0.5, 0.5)`
    - hex RGB or RGBA string, e.g.: `"#9f2925"`

In [ ]:
# CHANGE HERE. In this example, we have 8 different temperature ranges (in Celsius).
#   The colors are strings in hex RGB format.
#   You can define any colors and temperature ranges you want.
COLOR_CHART = {
  (-11, -7): "#9f2925",
  (-6, -2): "#d74b2a",
  (-1, 3): "#e76e2a",
  (4, 8): "#e18f3e",

  (9, 13): "#c0c5b3",
  (14, 18): "#84a15c",
  (19, 23): "#699751",
  (24, 28): "#244c36",
}

# COLOR_CHART = {
#   (-14, -11): "#9f2925",
#   (-10, -7): "#d74b2a",
#   (-6, -3): "#e76e2a",
#   (-2, 1): "#e18f3e",
#   (2, 5): "#e7a995",
#   (6, 9): "#e0c8be",

#   (10, 13): "#ccd8d6",
#   (14, 17): "#c0c5b3",
#   (18, 21): "#84a15c",
#   (22, 25): "#3aa544",
#   (26, 29): "#699751",
#   (30, 33): "#244c36",
#}

3. Think about how large the blanket should be

In [37]:
# CHANGE HERE. The column count determines how many hexagons span the width.
HEX_COLS = 17

# CHANGE HERE. Unit of the temperature
TEMP_UNIT = "°C"

# CHANGE HERE. If the figure should be saved.
SAVE_FIG = False

# CHANGE HERE. Folder to save the figures.
DST_DIR = "./"


4. Define utility functions

In [38]:
def get_color_for(temp: int) -> str:
  """Return the color assigned to this temperature value."""
  # Some temperatures are decimal numbers. However, we only defined integers in COLOR_CHART.
  temp = round(temp)

  for key in COLOR_CHART.keys():
    if temp >= key[0] and temp <= key[1]:
      return COLOR_CHART[key]

  # If the color is not found, use black.
  return "#000000"


def create_legend_handles():
  handles = []
  for k, v in COLOR_CHART.items():
    patch = mpatches.Patch(color=v, label=f"{k[0]}{TEMP_UNIT} - {k[1]}{TEMP_UNIT} ({v})")
    handles.append(patch)

  return handles


def calculate_hexagon_centers(num: int = 1, max_cols: int = 1) -> list:
  """Calculate the center points required to arrange the given number of hexagons.

  args:
    num: Number of hexagons to position.
    max_cols: Maximum number of colums.
  """
  # We want to draw at least one hexagon.
  if num < 1:
    return []

  # Predefined size of the hexagons.
  edge_length = np.sqrt(1/3)
  in_radius = 0.5

  only_place_hex_start, only_place_hex_start_end = (False, False)

  # Calculate the number of hexagons.
  num_rows_start = math.ceil(max_cols/2)
  num_start_hexagons = reduce(lambda x, y: x + y, [i * 2 + 1 for i in range(num_rows_start)])
  if num_start_hexagons >= num:
    num_start_hexagons = num
    # We have too few hexagons for the number of columns.
    only_place_hex_start = True

  num_rows_end = math.floor(max_cols/2)
  num_hex_end = reduce(lambda x, y: x + y, [i * 2 + 1 for i in range(num_rows_end)])
  if num_hex_end + num_start_hexagons >= num:
    num_hex_end = num - num_start_hexagons
    # We have too few hexagons for the number of columns.
    only_place_hex_start_end = True

  num_hex_middle = num - num_start_hexagons - num_hex_end

  # The first hexagon is in the upper left corner.
  first_hex_center = [0,0]
  diag_offset = first_hex_center

  hex_centers = []

  # First, calculate the center of the section that expands.
  centers, diag_offset, last_hex_center = calculate_start_hexagon_centers(num_start_hexagons, diag_offset, in_radius, edge_length, first_hex_center)
  hex_centers += centers

  if only_place_hex_start:
    return hex_centers

  if not only_place_hex_start_end:
    # Then, compute the coordinates for the center of the middle segment.
    centers, diag_offset = calculate_middle_hexagon_centers(num_hex_middle, diag_offset, in_radius, edge_length, last_hex_center)
    hex_centers += centers

  # Third, calculate the center of the reducing part.
  centers, diag_offset = calculate_end_hexagon_centers(num_hex_middle, diag_offset, in_radius, edge_length, last_hex_center)
  hex_centers += centers

  return hex_centers


def calculate_start_hexagon_centers(num_hex: int, diag_offset: list, in_radius: float, edge_length: float, first_hex_center: list):
  """Return centers, beginning of the next diagonal, and the center of the last hexagon."""
  centers = []
  cur_center = diag_offset
  last_hex_center = cur_center

  for i in range(num_hex):
    centers.append(cur_center)

    # If we reached the upper edge, start a new diagonal.
    if cur_center[1] >= first_hex_center[1]:
      diag_offset = calculate_next_diag_offset(diag_offset, in_radius)
      last_hex_center = cur_center
      cur_center = diag_offset
    else:
      # Move the next hexagon up and to the right on the diagonal.
      cur_center = calculate_next_center(cur_center, edge_length, in_radius)

  return centers, diag_offset, last_hex_center


def calculate_middle_hexagon_centers(num_hex: int, diag_offset: list, in_radius: float, edge_length: float, last_hex_center: list):
  """Return centers and beginning of the next diagonal."""
  centers = []
  cur_center = diag_offset
  row_completed = False

  for i in range(num_hex):
    centers.append(cur_center)

    # If the previous diagonal was not completed, complete it.
    if cur_center[0] >= last_hex_center[0]:
      diag_offset = calculate_next_diag_offset(diag_offset, in_radius)
      cur_center = diag_offset
      row_completed = True
    else:
      cur_center = calculate_next_center(cur_center, edge_length, in_radius)
      row_completed = False

  # Calculate the next diagonal to fill. This may lead to holes in the middle.
  if not row_completed:
    diag_offset = calculate_next_diag_offset(diag_offset, in_radius)

  return centers, diag_offset


def calculate_end_hexagon_centers(num_hex: int, diag_offset: list, in_radius: float, edge_length: float, last_hex_center: list):
  """Return centers and beginning of the next diagonal."""
  centers = []
  # We have to start two hexagonal up
  diag_offset = [diag_offset[0] + 3/2 * edge_length, diag_offset[1] + in_radius]
  diag_offset = [diag_offset[0] + 3/2 * edge_length, diag_offset[1] + in_radius]
  cur_center = diag_offset

  for i in range(num_hex):
    centers.append(cur_center)

    if cur_center[0] >= last_hex_center[0]:
      diag_offset = calculate_next_diag_offset(diag_offset, in_radius)
      # The starting point of the diagonal is two hexagons higher and further right.
      diag_offset = [diag_offset[0] + 3/2 * edge_length, diag_offset[1] + in_radius]
      diag_offset = [diag_offset[0] + 3/2 * edge_length, diag_offset[1] + in_radius]
      cur_center = diag_offset
    else:
      cur_center = calculate_next_center(cur_center, edge_length, in_radius)

  return centers, diag_offset

def calculate_next_center(prev_center: list, edge_length: float, in_radius: float):
  """Calculate next center on the diagonal."""
  return [prev_center[0] + 3/2 * edge_length, prev_center[1] + in_radius]

def calculate_next_diag_offset(diag_offset: list, in_radius: float):
  """Calculate the beginning of the next diagonal."""
  return [diag_offset[0], diag_offset[1] - 2 * in_radius]


5. Util functions to color the hexagons differently

In [39]:
def create_half_hexagon(color, offset, flip=False):
  """Create half hexagon with the center (0,0) and minimal diameter of 1."""
  center = [0 + offset[0], 0 + offset[1]]
  circum_radius = np.sqrt(1/3)
  in_radius = 0.5

  if flip:
    codes, verts = zip(*[
      # Calculate the point for the half side
        (Path.MOVETO, [center[0] + np.sqrt(3)/4,    center[1] + 0.25]),
        (Path.LINETO, [center[0] + circum_radius,   center[1]]),
        (Path.LINETO, [center[0] + circum_radius/2, center[1] - in_radius]),

        (Path.LINETO, [center[0] - circum_radius/2, center[1] - in_radius]),
        (Path.LINETO, [center[0] - np.sqrt(3)/4,    center[1] - 0.25]),
        (Path.CLOSEPOLY, [center[0] - np.sqrt(3)/4, center[1] - 0.25])])
  else:
    codes, verts = zip(*[
        (Path.MOVETO, [center[0] - circum_radius,   center[1]]),
        (Path.LINETO, [center[0] - circum_radius/2, center[1] + in_radius]),
        (Path.LINETO, [center[0] + circum_radius/2, center[1] + in_radius]),

        # Calculate the point for the half side
        (Path.LINETO, [center[0] + np.sqrt(3)/4, center[1] + 0.25]),
        (Path.LINETO, [center[0] - np.sqrt(3)/4, center[1] - 0.25]),
        (Path.CLOSEPOLY, [center[0] - np.sqrt(3)/4, center[1] - 0.25])])

  return PathPatch(Path(verts, codes), color=color)


def create_whole_hexagon(color, offset):
  """Create whole hexagon with the center (0,0) and minimal diameter of 1."""
  center = [0 + offset[0], 0 + offset[1]]

  return RegularPolygon(center, 6, radius=np.sqrt(1/3), orientation=90*math.pi/180, color=color)


def create_nested_hexagon(color, offset, inner=False):
  """Make inner and outer hexagon with the center (0,0) and minimal diameter of 1."""
  center = [0 + offset[0], 0 + offset[1]]
  circum_radius = np.sqrt(1/3)
  in_radius = 0.5

  factor = 1
  if inner:
    # The inner hexagon is twice as small.
    factor = 1/2

  codes, verts = zip(*[
    (Path.MOVETO, [center[0] - circum_radius * factor,    center[1]]),
    (Path.LINETO, [center[0] - circum_radius/2 * factor,  center[1] + in_radius * factor]),
    (Path.LINETO, [center[0] + circum_radius/2 * factor,  center[1] + in_radius * factor]),

    (Path.LINETO, [center[0] + circum_radius * factor,    center[1]]),
    (Path.LINETO, [center[0] + circum_radius/2 * factor,  center[1] - in_radius * factor]),
    (Path.LINETO, [center[0] - circum_radius/2 * factor,  center[1] - in_radius * factor]),
    (Path.CLOSEPOLY, [center[0] - circum_radius * factor, center[1]])])

  return PathPatch(Path(verts, codes), color=color)

6. Read the temperature data

In [ ]:
# CHANGE HERE the file name
FILE_NAME = 'daily-historical-weather.csv'

# For a JSON file format
def read_json_file():
  with open("daily-historical-weather.json", "r") as f:
      data = json.load(f)

  min_max_temps = []
  avg_temps = []

  for day in data["data"]:
      max_temp = day["max_temp"]
      min_temp = day["min_temp"]
      avg_temp = day["temp"]

      min_max_temps.append((min_temp, max_temp))
      avg_temps.append(avg_temp)

  return min_max_temps, avg_temps


def read_csv_file():
  min_max_temps = []
  avg_temps = []

  with open(FILE_NAME, 'r') as f:
    reader = csv.reader(f)
    next(reader, None)  # skip the headers

    for row in reader:
          min_max_temps.append((float(row[0]), float(row[1])))
          avg_temps.append(float(row[2]))

  return min_max_temps, avg_temps


min_max_temps, avg_temps = read_csv_file()

In [ ]:
print(min(avg_temps))
print(max(avg_temps))

print(round(min(avg_temps)))
print(round(max(avg_temps)))

def my_func(a):
  return round(a)

avg_temps_list = map(my_func, avg_temps)

avg_temps_dict = defaultdict(int)
for el in avg_temps_list:
  avg_temps_dict[el] += 1

fig, ax = plt.subplots(1)
ax.set_ylabel('count')
ax.set_title('Average temperature distribution')
ax.legend(title='Count')

plt.axis('on')
plt.bar(avg_temps_dict.keys(), avg_temps_dict.values(), color='g')
plt.show()

## Design #1: Split Diagonal Hexagons

In [ ]:
hex_centers = calculate_hexagon_centers(num = len(min_max_temps), max_cols = HEX_COLS)

fig, ax = plt.subplots(figsize=(23, 16))
ax.set_aspect('equal')

for hex, min_max_temp in zip(hex_centers, min_max_temps):
  min_temp_color = get_color_for(min_max_temp[0])
  max_temp_color = get_color_for(min_max_temp[1])
  lower_half = create_half_hexagon(min_temp_color, hex, flip=True)
  upper_half = create_half_hexagon(max_temp_color, hex)

  ax.add_artist(lower_half)
  ax.add_artist(upper_half)
ax.set_xlim(-1, 15)
ax.set_ylim(-22, 1)

plt.axis('off')
handles = create_legend_handles()
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), handles=handles, title="Color chart")

if SAVE_FIG:
  save_to_filename = SysPath(DST_DIR) / "design1.pdf"
  plt.savefig(save_to_filename, dpi=300, format="pdf", bbox_inches='tight')
plt.show()

## Design #2: Solid Hexagons

In [ ]:
hex_centers = calculate_hexagon_centers(num = len(avg_temps), max_cols = HEX_COLS)

fig, ax = plt.subplots(figsize=(23, 16))
ax.set_aspect('equal')

for hex, avg_temp in zip(hex_centers, avg_temps):
  temp_color = get_color_for(avg_temp)
  whole_hex = create_whole_hexagon(temp_color, hex)

  ax.add_artist(whole_hex)
ax.set_xlim(-1, 15)
ax.set_ylim(-22, 1)

plt.axis('off')
handles = create_legend_handles()
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), handles=handles, title="Color chart")

if SAVE_FIG:
  save_to_filename = SysPath(DST_DIR) / "design2.pdf"
  plt.savefig(save_to_filename, dpi=300, format="pdf", bbox_inches='tight')
plt.show()

## Design #3: Nested Hexagons

In [ ]:
hex_centers = calculate_hexagon_centers(num = len(min_max_temps), max_cols = HEX_COLS)

fig, ax = plt.subplots(figsize=(23, 16))
ax.set_aspect('equal')

for hex, min_max_temp in zip(hex_centers, min_max_temps):
  min_temp_color = get_color_for(min_max_temp[0])
  max_temp_color = get_color_for(min_max_temp[1])
  outer_hex = create_nested_hexagon(max_temp_color, hex)
  # Inner hex is just drawn above the larger hexagon
  inner_hex = create_nested_hexagon(min_temp_color, hex, inner=True)

  ax.add_artist(outer_hex)
  ax.add_artist(inner_hex)

ax.set_xlim(-1, 15)
ax.set_ylim(-22, 1)

plt.axis('off')
handles = create_legend_handles()
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), handles=handles, title="Color chart")

if SAVE_FIG:
  save_to_filename = SysPath(DST_DIR) / "design3.pdf"
  plt.savefig(save_to_filename, dpi=300, format="pdf", bbox_inches='tight')
plt.show()